# Creating protein_coding_transcripts

In [18]:
!head protein_coding.gtf -n20

chr1	HAVANA	gene	65419	71585	.	+	.	gene_id "ENSG00000186092.6"; gene_type "protein_coding"; gene_name "OR4F5"; level 2; hgnc_id "HGNC:14825"; havana_gene "OTTHUMG00000001094.1";
chr1	HAVANA	transcript	65419	71585	.	+	.	gene_id "ENSG00000186092.6"; transcript_id "ENST00000641515.2"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-202"; level 2; protein_id "ENSP00000493376.2"; hgnc_id "HGNC:14825"; tag "RNA_Seq_supported_partial"; tag "basic"; havana_gene "OTTHUMG00000001094.1"; havana_transcript "OTTHUMT00000003223.1";
chr1	HAVANA	exon	65419	65433	.	+	.	gene_id "ENSG00000186092.6"; transcript_id "ENST00000641515.2"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-202"; exon_number 1; exon_id "ENSE00003812156.1"; level 2; protein_id "ENSP00000493376.2"; hgnc_id "HGNC:14825"; tag "RNA_Seq_supported_partial"; tag "basic"; havana_gene "OTTHUMG00000001094.1"; havana_transcript "OTT

In [19]:
!awk -v OFS="\t" '{if ($3 == "transcript" || $3 == "exon" || $3 == "CDS" || $3 == "UTR") print $1, $3, $4 - 1, $5, $7}' protein_coding.gtf > protein_coding_transcripts.bed
!cat protein_coding_transcripts.bed | head

chr1	transcript	65418	71585	+
chr1	exon	65418	65433	+
chr1	exon	65519	65573	+
chr1	CDS	65564	65573	+
chr1	exon	69036	71585	+
chr1	CDS	69036	70005	+
chr1	UTR	65418	65433	+
chr1	UTR	65519	65564	+
chr1	UTR	70005	71585	+
chr1	transcript	69054	70108	+
cat: write error: Broken pipe


In [20]:
f = open('protein_coding_transcripts.bed')
protein_coding_transcripts = f.readlines()
protein_coding_transcripts

['chr1\ttranscript\t65418\t71585\t+\n',
 'chr1\texon\t65418\t65433\t+\n',
 'chr1\texon\t65519\t65573\t+\n',
 'chr1\tCDS\t65564\t65573\t+\n',
 'chr1\texon\t69036\t71585\t+\n',
 'chr1\tCDS\t69036\t70005\t+\n',
 'chr1\tUTR\t65418\t65433\t+\n',
 'chr1\tUTR\t65519\t65564\t+\n',
 'chr1\tUTR\t70005\t71585\t+\n',
 'chr1\ttranscript\t69054\t70108\t+\n',
 'chr1\texon\t69054\t70108\t+\n',
 'chr1\tCDS\t69090\t70005\t+\n',
 'chr1\tUTR\t69054\t69090\t+\n',
 'chr1\tUTR\t70005\t70108\t+\n',
 'chr1\ttranscript\t450702\t451697\t-\n',
 'chr1\texon\t450702\t451697\t-\n',
 'chr1\tCDS\t450742\t451678\t-\n',
 'chr1\tUTR\t450702\t450742\t-\n',
 'chr1\tUTR\t451678\t451697\t-\n',
 'chr1\ttranscript\t685678\t686673\t-\n',
 'chr1\texon\t685678\t686673\t-\n',
 'chr1\tCDS\t685718\t686654\t-\n',
 'chr1\tUTR\t685678\t685718\t-\n',
 'chr1\tUTR\t686654\t686673\t-\n',
 'chr1\ttranscript\t923927\t939291\t+\n',
 'chr1\texon\t923927\t924948\t+\n',
 'chr1\tCDS\t924431\t924948\t+\n',
 'chr1\texon\t925921\t926013\t+\n',
 'chr

# Making classes.
## Chromosome, Transcript, Intron, Exon, CDS, UTR

In [21]:
class Chromosome:
    def __init__(self, nm): #string (format chr1, chr2, chrx, etc.)
        self.name = nm #string
        self.transcripts = []
    def show(self):
        show = str(str(self.name) + ": ")
        show += str(len(self.transcripts)) + " transcripts"
        print(show)
    def add_transcript(self, s, e, strand):
        self.transcripts.append(Transcript(s, e, strand))

In [22]:
class Transcript:
    def __init__(self, s, e, posneg): #int, int, string
        self.start = s #int
        self.end = e #int
        self.strand = (posneg == "+") #boolean
        self.exon_intron = []
        self.cds_start = -1 #smaller number, regardless of whether it's 5' or 3'
        self.cds_end = -1    #larger number, regardless of whether it's 3' or 5'
    def show(self):
        show = str("transcript: ")
        show += str(self.start) + " "      #start position
        show += str(self.end) + " "        #end position
        show += str(self.strand) + " "     #positive or negative strand
        show += str(len(self.exon_intron)) #how many exons
        print(show)
    def add_exon(self, s, e):
        self.exon_intron.append(Exon(s, e, self.strand))
        self.sort()
    def add_intron(self, s, e):
        self.exon_intron.append(Intron(s, e, self.strand))
        self.sort()
    def sort(self):
        self.exon_intron.sort(key=lambda x: x.start)
    def setCdsPos(self, exonIndex): #assumes there aren't any UTRs
        if (not self.exon_intron[exonIndex].has_cds()):
            return
        exon_cds_start = self.exon_intron[exonIndex].cds_utr[0].start
        exon_cds_end = self.exon_intron[exonIndex].cds_utr[0].end
        if (self.cds_start == -1 or self.cds_start > exon_cds_start):
            self.cds_start = exon_cds_start
        if (self.cds_end == -1 or self.cds_end < exon_cds_end):
            self.cds_end = exon_cds_end
    def has_intron(self):
        for x in self.exon_intron:
            if (type(x) == Intron):
                #print("there's an intron")
                return True
        #print("  no introns!")
        return False
    def has_utr(self):
        for x in self.exon_intron:
            if x.has_utr():
                return True
        return False

In [23]:
class Intron:
    def __init__(self, s, e, trnscrpt_strand): #int, int, boolean
        self.start = s #int
        self.end = e #int
        self.strand = trnscrpt_strand #boolean
    def show(self):
        show = str("intron: ")
        show += str(self.start) + " "      #start position
        show += str(self.end) + " "        #end position
        show += str(self.strand)           #positive or negative strand
        print(show)
    def has_utr(self):
        return False
    def has_cds(self):
        return False

In [24]:
class Exon:
    def __init__(self, s, e, trnscrpt_strand): #int, int, boolean
        self.start = s #int
        self.end = e #int
        self.strand = trnscrpt_strand #boolean
        self.cds_utr = []
    def show(self):
        show = str("exon: ")
        show += str(self.start) + " "      #start position
        show += str(self.end) + " "        #end position
        show += str(self.strand) + " "     #positive or negative strand
        show += str(len(self.cds_utr))     #how many CDSs and UTRs
        print(show)
    def add_cds(self, s, e):
        self.cds_utr.append(Cds(s, e, self.strand))
        self.sort()
    def add_utr(self, s, e, is_start):
        self.cds_utr.append(Utr(s, e, self.strand, is_start)) #start = UTR index < CDS index (5' if pos, 3' if neg)
        self.sort()
    def sort(self):
        self.cds_utr.sort(key=lambda x: x.start)
    def has_cds(self):
        for x in self.cds_utr:
            if (type(x) == Cds):
                return True
        return False
    def has_utr(self):
        for x in self.cds_utr:
            if (type(x) == Utr):
                return True
        return False

In [25]:
class Cds:
    def __init__(self, s, e, exon_strand): #int, int, boolean
        self.start = s #int
        self.end = e #int
        self.strand = exon_strand #boolean
    def show(self):
        show = str("CDS: ")
        show += str(self.start) + " "       #start position
        show += str(self.end) + " "        #end position
        show += str(self.strand)           #positive or negative strand
        print(show)

In [26]:
class Utr:
    def __init__(self, s, e, exon_strand, is_start): #int, int, boolean, boolean
        self.start = s #int
        self.end = e #int
        self.strand = exon_strand #boolean
        self.is_five = is_start                #True = 5' UTR, False = 3' UTR
        if (not self.strand):                  #if it is on the negative strand
            self.is_five = not self.is_five    #flip
    def show(self):
        if(self.is_five):
            show = str("5' ")
        else:
            show = str("3' ")
        show += str("UTR: ")
        show += str(self.start) + " "      #start position
        show += str(self.end) + " "        #end position
        show += str(self.strand) + " "     #positive or negative strand
        print(show)

In [27]:
example_transcript1 = Transcript(1, 100, "+")
example_transcript2 = Transcript(30, 600, "-")

example_transcript1.show()
example_transcript2.show()

print(example_transcript1)
print(example_transcript2)

print()
print("-------------")
print()

example_transcript1.add_exon(2, 10)
example_transcript1.add_exon(21, 22)
example_transcript1.add_exon(40, 50)
example_transcript2.add_exon(99, 580)

example_transcript1.show()
example_transcript2.show()

for i in range(3):
    print("example_transcript1")
    example_transcript1.exon_intron[i].show()
print("example_transcript2")
example_transcript2.exon_intron[0].show()

transcript: 1 100 True 0
transcript: 30 600 False 0

-------------

transcript: 1 100 True 3
transcript: 30 600 False 1
example_transcript1
exon: 2 10 True 0
example_transcript1
exon: 21 22 True 0
example_transcript1
exon: 40 50 True 0
example_transcript2
exon: 99 580 False 0


In [28]:
example_chromosome1 = Chromosome("chr1")
example_chromosome5 = Chromosome("chr5")

example_chromosome1.show()
example_chromosome5.show()

print()
print("-------------")
print()

example_chromosome1.add_transcript(11, 20, "+")

example_chromosome1.show()
example_chromosome5.show()

print()
print("-------------")
print()

test = example_chromosome1.transcripts[0]
print(example_chromosome1.transcripts[0])
print(test)
test.show()

chr1: 0 transcripts
chr5: 0 transcripts

-------------

chr1: 1 transcripts
chr5: 0 transcripts

-------------

transcript: 11 20 True 0


# Filling chr_list

### chromosomes, exons, CDSs

In [29]:
#note to self: you can use == to compare strings!!!
#this is a bit clunky.. maybe add extra methods in classes to make below cells better?

In [30]:
chr_list = []
chr_ind = -1
trn_ind = -1
exn_ind = -1
for x in protein_coding_transcripts:
    temp = x.split()
    if (chr_ind == -1 or temp[0] != chr_list[chr_ind].name): #if it's the next chromosome
        #create new chromosome in chr_list, reset transcript & exon indices
        if ("1" <= temp[0][3:4] and temp[0][3:4] <= "9"): #if it's a number
            if ((int(temp[0][3:]) < 1 or 22 < int(temp[0][3:]))):
                continue
        elif (temp[0] != "chrX" and temp[0] != "chrY"): #the rest are letters
            continue
        chr_list.append(Chromosome(temp[0]))
        chr_ind += 1
        trn_ind = -1
    if (temp[1] == 'transcript'): #if it's a transcript
        #add new transcript to current chromosome, reset exon index
        chr_list[chr_ind].add_transcript(int(temp[2]), int(temp[3]), temp[4])
        trn_ind += 1
        exn_ind = -1
    elif (temp[1] == 'exon'): #if it's an exon
        #add new exon to current transcript
        chr_list[chr_ind].transcripts[trn_ind].add_exon(int(temp[2]), int(temp[3]))
        exn_ind += 1
    elif (temp[1] == 'CDS'): #if it's a CDS
        #add new transcript to current exon
        chr_list[chr_ind].transcripts[trn_ind].exon_intron[exn_ind].add_cds(int(temp[2]), int(temp[3]))
    elif (temp[1] == 'UTR'): # if it's a UTR
        #add new UTR to whatever exon it's in
        currTranscript = chr_list[chr_ind].transcripts[trn_ind]
        currTranscript.setCdsPos(exn_ind)
        for exn in currTranscript.exon_intron: #assumes no introns yet
            if (exn.start <= int(temp[3])):
                if (int(temp[3]) <= exn.end):
                    currExon = exn
                    break
        
        if (int(temp[3]) < currTranscript.cds_start): #if UTR position is smaller than CDS position
            currExon.add_utr(temp[2], temp[3], True)
        elif (int(temp[2]) > currTranscript.cds_end): #if UTR position is larger than CDS position
            currExon.add_utr(temp[2], temp[3], False)
        else:
            print("something's wrong")

for chrm in chr_list:
    chrm.show()

TypeError: '<' not supported between instances of 'str' and 'int'

### UTRs

In [ ]:
#for each exon, whatever's not an exon is a UTR
#redo 5' 3' identification section

In [ ]:
#checks to make sure that none of the exons have more than one CDS (would be a problem in the next block of code)

plus = 0 #number of exons with more than 1 CDS
ones = 0 #number of exons with 1 CDS
zero = 0 #number of exons with 0 CDSs
for chrm in chr_list:
    for trn in chrm.transcripts:
        for exn in trn.exon_intron:
            if (type(exn) != Exon):
                continue
            if (len(exn.cds_utr) == 1):
                ones += 1
            elif (len(exn.cds_utr) < 1):
                zero += 1
            else:
                plus += 1
print("plus " + str(plus))
print("ones " + str(ones))
print("zero " + str(zero))

In [ ]:
#chr_list[0].transcripts[0]
#exon	65418	65433	+
#exon	65519	65573	+
#CDS	65564	65573	+
#exon	69036	71585	+
#CDS	69036	70005	+
#UTR	65418	65433	+	this is what needs to be added
#UTR	65519	65564	+	this is what needs to be added
#UTR	70005	71585	+	this is what needs to be added

#for each chromosome in chr_list:
#    for each transcript in chromosome:
#        save CDS start and end points
#        for each exon in transcript:
#            create variables for possible UTR start end points (for now, save as exon start end points)
#            if exon has CDS:
#                if exon.CDS start != exon start:
#                    save UTR1 end as CDS start
#                if exon.CDS end != exon end:
#                    save UTR2 start as CDS end
#            numUTRS = 2
#            if the 2 UTRs are the same:
#                numUTRs = 1
#            if UTR start >= UTR end: #probably not > but just in case
#                continue
#            if UTR end <= CDS start: #5' UTR if +, 3' UTR if - ("start UTR")
#                exn.add_utr(UTR start, UTR end, True) #0-indexed
#            else: #3' UTR if +, 5' UTR if - ("end UTR")
#                exn.add_utr(UTR start, UTR end, False) #0-indexed

In [ ]:
#the code assumes that the transcript is on the positive strand, utr constructor handles negative strand (I think)
#only one CDS per exon, and no UTRs yet (make something that checks this), so cds_utr[0] must be the only CDS
#doesn't take care of cases where 1 exon contains 2 UTRs

for chrm in chr_list:
    if (chrm == chr_list[2]):
        break
    chrm.show()
    for trn in chrm.transcripts:
        if (trn == chrm.transcripts[100]):
            break
        if (trn.has_utr()): #don't add UTRs if there already are UTRs
            exn.sort()
            continue
        trn.show()
        cds_start = -1
        cds_end = -1
        for tempExon in trn.exon_intron:
            if (cds_start == -1 and tempExon.has_cds()):
                cds_start = tempExon.cds_utr[0].start #second comment line applies to this
            if (tempExon.has_cds()):
                cds_end = tempExon.cds_utr[0].end     #second comment line applies to this
        print(str(cds_start) + " " + str(cds_end))
        for i in range(len(trn.exon_intron)):
            exn = trn.exon_intron[i]
            exn.show()
            utr_pos = [[exn.start, exn.end], [exn.start, exn.end]]
            if exn.has_cds():
                utr_pos[0][1] = exn.cds_utr[0].start  #second comment line applies to this
                utr_pos[1][0] = exn.cds_utr[0].end    #second comment line applies to this
            numUTRs = 2
            if (utr_pos[0] == utr_pos[1]):
                numUTRs = 1
            for j in range(numUTRs):
                print(utr_pos[j])
                if (utr_pos[j][0] >= utr_pos[j][1]): #probably not > but just in case
                    print("no UTR to add")
                    continue
                if (utr_pos[j][1] <= cds_start): #5' UTR if +, 3' UTR if - ("start UTR")
                    print("start UTR")
                    exn.add_utr(utr_pos[j][0], utr_pos[j][1], True) #0-indexed
                else: #3' UTR if +, 5' UTR if - ("end UTR")
                    print("end UTR")
                    exn.add_utr(utr_pos[j][0], utr_pos[j][1], False) #0-indexed

In [ ]:
chr_list[1].show()
for thingy in chr_list[1].transcripts[0].exon_intron:
    print()
    thingy.show()
    for thing in thingy.cds_utr:
        thing.show()

In [ ]:
#current issue: negative strand genes are ordered backwards
#solution 1: fix the order when creating the bed file (or the list, or both)
#solution 2: leave the data alone but deal with the issue in the above code

In [ ]:
for chrm in chr_list:
    #chrm.show()
    for trn in chrm.transcripts:
        for exn in trn.exon_intron:
            if (exn.has_utr()): #don't add UTRs if there already are UTRs
                exn.sort()
                continue
            if (len(exn.cds_utr) == 0): #whole exon = UTR
                exn.add_utr(exn.start, exn.end, True) #if UTR spans entirety of exon, is it 5' or 3'?
                continue
            cds = exn.cds_utr[0]
            if (exn.start < cds.start):
                exn.add_utr(exn.start, cds.start, True) #0-indexed
            if (cds.end < exn.end):
                exn.add_utr(exn.end, exn.start, False) #0-indexed
            exn.sort()

chr_list[0].transcripts[1].exon_intron[0].cds_utr

### Introns

In [ ]:
#for each transcript, whatever's not an exon is an intron

In [ ]:
chr_list[1].transcripts[0].show()

In [ ]:
for thing in chr_list[1].transcripts[0].exon_intron:
    print(str(type(thing)) + " " + str(thing.start) + " " + str(thing.end))

In [ ]:
#for chrm in chr_list:
for chrm in chr_list:
    #if (chrm == chr_list[2]):
        #break
    #chrm.show()
    for trn in chrm.transcripts:
        #if (trn == chrm.transcripts[1]):
            #break
        if (trn.has_intron()): #don't add introns if there already are introns
            #trn.sort()
            #print("  yes intron")
            continue
        #print("  moving on")
        prev = trn.start
        exon_list_copy = trn.exon_intron.copy()
        for exn in exon_list_copy:
            #print("    next exon")
            curr = exn.start
            #print("    prev = " + str(prev) + " curr = " + str(curr))
            if (prev < curr):
                #print("    adding intron from " + str(prev) + " to " + str(curr))
                trn.add_intron(prev, curr) #0-indexed
            prev = exn.end
        #print("  ended for loop")
        #print("    prev = " + str(prev) + " trn.end = " + str(trn.end))
        if (prev < trn.end):
            #print("    adding intron from " + str(prev) + " to " + str(trn.end))
            trn.add_intron(prev, trn.end) #0-indexed
        #trn.sort()

chr_list[1].show()
for thing in chr_list[1].transcripts[0].exon_intron:
    print(str(type(thing)) + " " + str(thing.start) + " " + str(thing.end))

In [ ]:
chr_list[0].show() #gene on positive strand
for thingy in chr_list[0].transcripts[0].exon_intron:
    thingy.show()
print()
chr_list[1].show() #gene on negative strand
for thingy in chr_list[1].transcripts[0].exon_intron:
    thingy.show()

In [ ]:
endpoints = []
for x in protein_coding_transcripts:
  endpoints.append(int(x.split()[2]))
  endpoints.append(int(x.split()[3]))
endpoints

In [ ]:
endpoints = list(dict.fromkeys(endpoints))
endpoints.sort()
endpoints

In [ ]:
#OMG I JUST REALIZED THAT THE CELL BELOW IGNORES CHROMOSOMES!!!

In [ ]:
#make a list with segment start point, segment end point, 
#number CDS, number start UTR, number end UTR, number intron, 
#percent CDS, percent start UTR, percent end UTR, percent intron
#note: number columns are 0, percent columns should be blank for now (placeholder 0)
#note: will turn this into a bed file later

segments_list = []

for i in range(int(len(endpoints) -1)):
  segments_list.append([endpoints[i], endpoints[i + 1], 0, 0, 0, 0, 0, 0, 0, 0])
segments_list

In [ ]:
#for each segment (a.k.a. item in list from previous cell):
###pick a point in the segment
###for each transcript:
#####for each exon in the transcript:
#######if it is between the endpoints of the exon:
#########exon+=1 (it is an exon for this transcript)
#######else
#########if it is between the endpoints of the transcript:
###########intron+=1 (it is an intron for this transcript)

#note: this no longer fully reflects the below code

In [ ]:
import time

In [ ]:
#The cell below takes too long to run. I calculated it to take rougghly 40 hours
#(I ran it for 100 lines and found how many lines total)

In [ ]:
start = time.time()

for x_ind in range(len(segments_list)):
    x = segments_list[x_ind]
    point = x[0] + 1
    #print(point)
    for y in transcripts_list:
        exon = False
        for j in range(2, len(y)):
            if (y[j][0] < point) and (point < y[j][1]): #exon
                if (len(y[j]) > 2 and y[j][2][0] < point) and (point < y[j][2][1]): #CDS
                    x[2] += 1
                elif (len(y[j]) > 2 and y[j][2][0] > point): #start UTR, since the segment is part of the exon but before the CDS
                    x[3] += 1
                else: #end UTR
                    x[4] += 1
                    exon = True
                    #print("point " + str(point) + " is in an exon when j = " + str(j))
        if not exon and (y[0] < point) and (point < y[1]): #intron
            x[5] += 1
            #print("point " + str(point) + " is in an intron.")
    if (x_ind % 10 == 0): #checking to make sure it runs
        print(x_ind)
    if (x_ind == 100):
        break

print(time.time() - start)

segments_list

In [ ]:
for x in segments_list:
    if ((x[2] + x[3] + x[4] + x[5]) == 0):
        continue;
    x[6] = x[2] / (x[2] + x[3] + x[4] + x[5])
    x[7] = x[3] / (x[2] + x[3] + x[4] + x[5])
    x[8] = x[4] / (x[2] + x[3] + x[4] + x[5])
    x[9] = x[5] / (x[2] + x[3] + x[4] + x[5])
segments_list